In [ ]:
from google.colab import drive
drive.flush_and_unmount()



Drive not mounted, so nothing to flush and unmount.


In [ ]:
!git clone https://github.com/LaasyaGujjula/dataset
!cd dataset

Cloning into 'dataset'...
remote: Enumerating objects: 173211, done.
remote: Total 173211 (delta 0), reused 0 (delta 0), pack-reused 173211
Receiving objects: 100% (173211/173211), 1.52 GiB | 33.42 MiB/s, done.
Updating files: 100% (190335/190335), done.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

In [ ]:
base_dir = '/content/dataset'
train_dir = os.path.join(base_dir, 'Train')
val_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Test')

batch_size = 32
img_size = (128, 128)  # Reduced image size for faster processing

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = Flatten()(mobilenet_base.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
mobilenet_model = Model(inputs=mobilenet_base.input, outputs=output)


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
9406464/9406464 [==============================] - 0s 0us/step


In [ ]:

mobilenet_model = Model(inputs=mobilenet_base.input, outputs=output)

for layer in mobilenet_base.layers:
    layer.trainable = False

mobilenet_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Unfreeze the last 20 layers for fine-tuning
for layer in mobilenet_base.layers[-20:]:
    layer.trainable = True

In [ ]:
epochs = 1  # Reduced number of epochs for quicker training

mobilenet_model.fit(train_generator, epochs=epochs, validation_data=val_generator)

4376/4376 [==============================] - 731s 165ms/step - loss: 0.4875 - accuracy: 0.7680 - val_loss: 0.4380 - val_accuracy: 0.7952


In [ ]:

mobilenet_loss, mobilenet_acc = mobilenet_model.evaluate(test_generator)

print(f'MobileNetV2 Test Accuracy: tg{mobilenet_acc}')

341/341 [==============================] - 14s 42ms/step - loss: 0.5311 - accuracy: 0.7342
MobileNetV2 Test Accuracy: tg0.7341586351394653


In [ ]:
def predict_image(model, img_path):
    img = load_img(img_path, target_size=(128, 128))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print("The image is predicted to be real.")
    else:
        print("The image is predicted to be fake.")

# Example usage
image_path = '/content/dataset/Test/Fake/fake_100.jpg'
predict_image(mobilenet_model, image_path)

1/1 [==============================] - 0s 39ms/step
The image is predicted to be fake.
